In [1]:
import spacy
import pandas as pd
import numpy as np

import solrhandler as sh
import clusterer as cls
import topicdeterminator as td
import chatbot as cb

In [2]:
#query = "anmeldung"
#target_service = 120686

Evaluierungspipeline

In [3]:
# Import eval.csv
import os
file = os.path.realpath("../../Evaluierung/eval.csv")
eval = pd.read_csv(file, sep = ";")
eval = eval.head(5)
eval

,documentId,file,dialogId,initialQuestion,suggestionCount,directlyFound,name
0,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung
1,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung
2,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...
3,329421,20220131--quantEvalAllExcel.xlsx,19,Quarantäne Krankschreibung vom Arzt,6,True,Erstattung nach Infektionsschutzgesetz bei Tät...
4,120914,20220131--quantEvalAllExcel.xlsx,23,Termin für Zulassung Gebrauchtwagen,6,True,Zulassung eines Fahrzeuges mit auswärtigem Ken...


Funktionen

In [10]:
def findCorrectAnswer(chatbot,targetService):

    # Find Target Cluster
    clusteredColumn = chatbot.clusterer.getClusteredColumn()
    service = chatbot.df.loc[chatbot.df["id"]==str(targetService)]
    if len(service[clusteredColumn].values) == 0:
        return None

    targetCluster = service[clusteredColumn].values[0]

    if targetCluster == chatbot.getSelectedClusterForQuestion():
        return True

    else:
        return False

def getRank(chatbot, targetService):
    temp = chatbot.df#.reset_index()
    rank = temp.index[temp["id"]==str(targetService)].values[0]
    return rank

Logging

In [15]:
class Logger:
    
    def __init__(self, chatbot, eval_df):
        self.liste = []
        self.chatbot = chatbot
        self.eval_df = eval_df
    
    def getDataFrame(self):
        pass
    
    def log(self, i, t, answer = None, target_service = None): 
        row = ()
        if answer is not None:
            nResults = len(self.chatbot.df.index)
            question = self.chatbot.generateQuestion()
            answer = answer

            rank = getRank(self.chatbot,target_service)# getRank(target_service)
            
            row = (i,t,nResults, question, answer, rank)
        else:
            #nInitialResults = len(chatbot.df.index)
            #initialRank = np.nan
            row = (i,t,None, None, None, None)
        self.liste.append(row)
        #print(self.liste)

Execution

In [20]:
# Initialisierung Komponenten Chatbot
solrhandler = sh.SolrHandler(20)
clusterer = cls.Clusterer()
tpc_dterminator = td.TopicDeterminator()
chatbot = cb.Chatbot(solrhandler,clusterer,tpc_dterminator)

lg = Logger(chatbot,eval)   

for i in [0,1,2,3,4]:#np.arange(0,len(eval.index)-1):
    
    # fun
    query = eval["initialQuestion"].values[i]#"Ummeldung"
    target_service = eval["documentId"].values[i]
    chatbot.initialQuery(query)
    
    # TODO
    t = 0
    answer = findCorrectAnswer(chatbot, target_service)
    lg.log(i,t, answer, target_service)
    if answer is None:
        continue
    
    
    while not chatbot.refineResultset(answer):
        # log
        t += 1
        
        answer = findCorrectAnswer(chatbot, target_service)
        if answer is None: 
            break
        lg.log(i, t, answer, target_service)
    t+=1
    #lg.log(i,t,answer)
    
        
log_df = pd.DataFrame(lg.liste, columns=["i","t","nResults", "question", "answer", "rank"])

C:\Users\morit\anaconda3\envs\chatbot\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\morit\anaconda3\envs\chatbot\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\morit\anaconda3\envs\chatbot\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\morit\anaconda3\envs\chatbot\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarni

In [21]:
log_df

,i,t,nResults,question,answer,rank
0,0,0,7.0,Geht es bei ihrem Anliegen um zulassung?,True,2.0
1,1,0,NaN,None,None,NaN
2,2,0,13.0,Geht es bei ihrem Anliegen um approbation?,False,3.0
3,2,1,6.0,Geht es bei ihrem Anliegen um sars?,True,3.0
4,3,0,10.0,Geht es bei ihrem Anliegen um approbation?,False,0.0
5,3,1,3.0,Geht es bei ihrem Anliegen um quarantäne?,True,0.0
6,4,0,20.0,Geht es bei ihrem Anliegen um kfz?,True,6.0


TEST

In [ ]:
chatbot.df